In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os
import csv
from tqdm import tqdm

import tensorflow as tf
from tensorflow import shape,math
from tensorflow.keras import Input,layers,Model
from tensorflow.keras.losses import mse,binary_crossentropy
from tensorflow.keras.utils import plot_model

import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
# import os
# import pandas as pd
# import numpy as np

# output_dir = os.path.join(".", "output")  # Path to the output directory
# subdirs = [f.path for f in os.scandir(output_dir) if f.is_dir()]
# subdirs.sort()

# # Create an empty 3D array to store the combined data
# combined_arr = np.empty((len(subdirs), 999, 10))

# # Loop through each subdirectory and load the CSV files
# for i, subdir in enumerate(subdirs):
#     action_filename = os.path.join(subdir, "action.csv")
#     obs_filename = os.path.join(subdir, "obs.csv")

#     # Load the action and obs CSV files
#     action_df = pd.read_csv(action_filename)
#     obs_df = pd.read_csv(obs_filename)

#     # Concatenate the DataFrames horizontally
#     combined_data = pd.concat([action_df, obs_df], axis=1)

#     # Convert combined_data to a 3D array and assign it to combined_arr
#     combined_arr[i] = np.reshape(combined_data.values, (999, 10))

# # Print the shape of combined_arr
# print(combined_arr.shape)


In [ ]:
# flattened_arr = combined_arr.reshape(10000, 9990)

In [ ]:
# flattened_arr.shape

In [ ]:
# # Rename dimensions to 'Seed' and 'Data'
# flattened_tensor.names = ("Seed", "Data")

# # Now flattened_tensor will have dimensions named 'Seed' and 'Data'
# print(flattened_tensor.shape)  # Output: torch.Size([10000, 9990])
# print(flattened_tensor)

In [ ]:
# len(flattened_tensor)

In [ ]:
import os
import pandas as pd
import numpy as np

output_dir = os.path.join(".", "output_bw")  # Path to the output directory
subdirs = [f.path for f in os.scandir(output_dir) if f.is_dir()]
subdirs.sort()

# Create an empty 3D array to store the combined data
combined_arr = np.empty((len(subdirs), 1000, 10))

# Loop through each subdirectory and load the CSV files
for i, subdir in enumerate(subdirs):
    action_filename = os.path.join(subdir, "action.csv")
    obs_filename = os.path.join(subdir, "obs.csv")

    # Load the action and obs CSV files
    action_df = pd.read_csv(action_filename,  header=None)
    obs_df = pd.read_csv(obs_filename,  header=None)

    # Concatenate the DataFrames horizontally
    combined_data = pd.concat([action_df, obs_df], axis=1)

    # Convert combined_data to a 3D array and assign it to combined_arr
    combined_arr[i-1] = np.reshape(combined_data.values, (1000, 10))

# Print the shape of combined_arr
print(combined_arr.shape)

In [ ]:
combined_df = np.array(combined_arr)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
## 정규화하기

# Flatten the combined_arr to 2D
flattened_arr = combined_arr.reshape(-1, combined_arr.shape[-1])

# Create a scaler object
scaler = MinMaxScaler()

# Fit the scaler and transform the data
normalized_arr = scaler.fit_transform(flattened_arr)

# Reshape the normalized data back to the original shape
normalized_arr = normalized_arr.reshape(combined_arr.shape)

# Print the shape of normalized_arr
print(normalized_arr.shape)

In [ ]:
combined_df = np.array(combined_arr)

In [ ]:
## 정규화 다시 되돌리기

# Flatten the normalized_arr to 2D
flattened_normalized_arr = normalized_arr.reshape(-1, normalized_arr.shape[-1])

# Use the inverse_transform method to transform the data back to its original state
original_arr = scaler.inverse_transform(flattened_normalized_arr)

# Reshape the original data back to the original shape
original_arr = original_arr.reshape(combined_arr.shape)

# Print the shape of original_arr
print(original_arr.shape)


In [ ]:
import os
import pandas as pd
from tqdm import tqdm  # Import tqdm

output_dir = os.path.join(".", "output_front")  # Path to the output directory
subdirs = [f.path for f in os.scandir(output_dir) if f.is_dir()]
combined_df = pd.DataFrame()

# Create a tqdm progress bar
pbar = tqdm(subdirs, desc="Processing subdirectories", unit="subdir")

for subdir in pbar:
    action_filename = os.path.join(subdir, "action.csv")
    obs_filename = os.path.join(subdir, "obs.csv")
    action_df = pd.read_csv(action_filename)
    obs_df = pd.read_csv(obs_filename)
    combined_data = pd.concat([action_df, obs_df], axis=1)
    if combined_data.shape == (999,10):
        combined_df = pd.concat([combined_df, combined_data], ignore_index=True)
    else:
        print("fail : ", subdir)

    # Update the progress bar
    pbar.set_postfix({"Processed subdirectories": subdir})

# Close the progress bar
pbar.close()



In [ ]:
import torch

combined_tensor = torch.from_numpy(normalized_arr)
print(combined_tensor.shape)
flattened_tensor = combined_tensor.permute(2, 0, 1).flatten(start_dim=1)

print(flattened_tensor.shape)


In [ ]:

# combined_df = np.array(combined_arr)
# combined_tensor = torch.from_numpy(combined_df)
# combined_tensor = combined_tensor.permute(2, 0, 1).flatten(start_dim=1)

# print(combined_tensor.shape[0])
# print(combined_tensor.shape[1])

# VAE code -2

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split

# Assuming combined_tensor is your data
# Convert the data to float32
dataset = TensorDataset(combined_tensor.float())

# Define the data loader
batch_size = 256  # adjust as necessary

# Split data into train, validation, and test
train_size = int(0.7 * len(dataset))  # 70% for training
valid_size = int(0.15 * len(dataset))  # 15% for validation
test_size = len(dataset) - train_size - valid_size  # 15% for testing

train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Hyperparameters
input_shape = combined_tensor.shape[1] * combined_tensor.shape[2]  # modify this to match your data
hidden_dim1 = 512  # modify as needed
hidden_dim2 = 256  # modify as needed
hidden_dim3 = 126  # modify as needed
latent_dim = 2  # modify as needed
lr = 1e-3  # learning rate


class VAE(nn.Module):
    def __init__(self, input_shape, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_shape, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim3),
            nn.ReLU(),
            nn.Linear(hidden_dim3, latent_dim * 2)  # mean and variance
        )

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim3),
            nn.ReLU(),
            nn.Linear(hidden_dim3, input_shape),
#             nn.Sigmoid()      ## 시그모이드를 뺴야됨. 왜냐면 애초데이터가 0과 1사이가 아니기때문
        )

    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu, log_var = h.chunk(2, dim=1)
        z = self.reparameterize(mu, log_var)
        x_reconstructed = self.decoder(z)
        return x_reconstructed, mu, log_var
    
# Model, optimizer, and loss function
model = VAE(input_shape, hidden_dim1, latent_dim)

optimizer = optim.NAdam(model.parameters(), lr=lr)
loss_fn = nn.MSELoss() #MSE로 바꿔보자.

def train(epoch, model, optimizer, loss_fn, train_loader):
    model.train()
    train_loss = 0
    total_recon_loss = 0
    total_kl_div = 0
    for i, batch in enumerate(train_loader):  # using train_loader instead of dataloader
        batch_data = batch[0]  # get the data from the batch

        optimizer.zero_grad()

        # Flatten the data
        batch_data = batch_data.view(batch_data.size(0), -1)

        reconstructed_batch, mu, log_var = model(batch_data)

        # Loss: reconstruction loss + KL divergence
        recon_loss = loss_fn(reconstructed_batch, batch_data)
        kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        loss = recon_loss + kl_divergence

        loss.backward()
        train_loss += loss.item()
        total_recon_loss += recon_loss.item()
        total_kl_div += kl_divergence.item()
        optimizer.step()

    avg_loss = train_loss / len(train_loader.dataset)
    avg_recon_loss = total_recon_loss / len(train_loader.dataset)
    avg_kl_div = total_kl_div / len(train_loader.dataset)
    print(f'====> Epoch: {epoch} Average loss: {avg_loss}, Recon Loss: {avg_recon_loss}, KL Div: {avg_kl_div}')

    return avg_loss, avg_recon_loss, avg_kl_div


In [ ]:
import matplotlib.pyplot as plt

# Lists to store losses for each epoch
avg_losses = []
avg_recon_losses = []
avg_kl_divs = []

# Training
n_epochs = 50  # modify as needed
for epoch in range(1, n_epochs + 1):
    avg_loss, avg_recon_loss, avg_kl_div = train(epoch, model, optimizer, loss_fn, train_loader)
    avg_losses.append(avg_loss)
    avg_recon_losses.append(avg_recon_loss)
    avg_kl_divs.append(avg_kl_div)

# Plot the losses
plt.figure(figsize=(7,5))
plt.plot(avg_losses, label='Average Loss')
plt.plot(avg_recon_losses, label='Reconstruction Loss')
plt.plot(avg_kl_divs, label='KL Divergence')
plt.title('Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


# VAE code -1

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_shape, hidden_dim1, hidden_dim2, hidden_dim3, latent_dim):
        super(Encoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_shape, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim3),
            nn.ReLU(),
            nn.Linear(hidden_dim3, latent_dim * 2)  # mean and variance
        )

    def forward(self, x):
        h = self.encoder(x)
        mu, log_var = h.chunk(2, dim=1)
        return mu, log_var


class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_shape):
        super(Decoder, self).__init__()

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim3),
            nn.ReLU(),
            nn.Linear(hidden_dim3, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, output_shape),
        )

    def forward(self, z):
        x_reconstructed = self.decoder(z)
        return x_reconstructed


class VAE(nn.Module):
    def __init__(self, input_shape, hidden_dim1, hidden_dim2, hidden_dim3, latent_dim):
        super(VAE, self).__init__()

        self.encoder = Encoder(input_shape, hidden_dim1, hidden_dim2, hidden_dim3, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim1, hidden_dim2, hidden_dim3, input_shape)

    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, log_var = self.encoder(x)
        z = self.reparameterize(mu, log_var)
        x_reconstructed = self.decoder(z)
        return x_reconstructed, mu, log_var

    
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split

# Assuming combined_tensor is your data
# Convert the data to float32
dataset = TensorDataset(combined_tensor.float())

# Define the data loader
batch_size = 512  # adjust as necessary

# Split data into train, validation, and test
train_size = int(0.7 * len(dataset))  # 70% for training
valid_size = int(0.15 * len(dataset))  # 15% for validation
test_size = len(dataset) - train_size - valid_size  # 15% for testing

train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Hyperparameters
input_shape = combined_tensor.shape[1] * combined_tensor.shape[2]  # modify this to match your data
hidden_dim1 = 512  # modify as needed
hidden_dim2 = 256  # modify as needed
hidden_dim3 = 24  # modify as needed
latent_dim = 2  # modify as needed
lr = 5e-5  # learning rate
n_epochs = 400  # modify as needed
beta = 0.2
    
# Model, optimizer, and loss function
model = VAE(input_shape, hidden_dim1, hidden_dim2, hidden_dim3, latent_dim)

optimizer = optim.RAdam(model.parameters(), lr=lr)  # Make sure you're using the correct optimizer
loss_fn = nn.MSELoss()  # And the correct loss function


def train(epoch, model, optimizer, loss_fn, train_loader):
    model.train()
    train_loss = 0
    total_recon_loss = 0
    total_kl_div = 0
    for i, batch in enumerate(train_loader):  # using train_loader instead of dataloader
        batch_data = batch[0]  # get the data from the batch
        
        optimizer.zero_grad()

        # Flatten the data
        batch_data = batch_data.view(batch_data.size(0), -1)

        reconstructed_batch, mu, log_var = model(batch_data)

        # Loss: reconstruction loss + KL divergence
        recon_loss = loss_fn(reconstructed_batch, batch_data)
        kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        loss = recon_loss + beta*kl_divergence

        loss.backward()
        train_loss += loss.item()
        total_recon_loss += recon_loss.item()
        total_kl_div += kl_divergence.item()
        optimizer.step()

    avg_loss = train_loss / len(train_loader.dataset)
    avg_recon_loss = total_recon_loss / len(train_loader.dataset)
    avg_kl_div = total_kl_div / len(train_loader.dataset)
    print(f'====> Epoch: {epoch} Average loss: {avg_loss}, Recon Loss: {avg_recon_loss}, KL Div: {avg_kl_div}')

    return avg_loss, avg_recon_loss, avg_kl_div

    
import matplotlib.pyplot as plt

# Lists to store losses for each epoch
avg_losses = []
avg_recon_losses = []
avg_kl_divs = []

# Training
for epoch in range(1, n_epochs + 1):
    avg_loss, avg_recon_loss, avg_kl_div = train(epoch, model, optimizer, loss_fn, train_loader)
    avg_losses.append(avg_loss)
    avg_recon_losses.append(avg_recon_loss)
    avg_kl_divs.append(avg_kl_div)

# Plot the losses
plt.figure(figsize=(7,5))
plt.plot(avg_losses, label='Average Loss')
plt.plot(avg_recon_losses, label='Reconstruction Loss')
plt.plot(avg_kl_divs, label='KL Divergence')
plt.title('Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
def evaluate(model, loss_fn, dataloader):
    model.eval()
    loss = 0
    total_recon_loss = 0
    total_kl_div = 0
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            batch_data = batch[0]  # get the data from the batch

            # Flatten the data
            batch_data = batch_data.view(batch_data.size(0), -1)

            reconstructed_batch, mu, log_var = model(batch_data)

            # Loss: reconstruction loss + KL divergence
            recon_loss = loss_fn(reconstructed_batch, batch_data)
            kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            loss += recon_loss + kl_divergence
            total_recon_loss += recon_loss.item()
            total_kl_div += kl_divergence.item()

    avg_loss = loss / len(dataloader.dataset)
    avg_recon_loss = total_recon_loss / len(dataloader.dataset)
    avg_kl_div = total_kl_div / len(dataloader.dataset)

    return avg_loss, avg_recon_loss, avg_kl_div

In [ ]:
evaluate(model, loss_fn, test_loader)

# VAE code -3 [10,1000000]

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_shape, hidden_dim1, hidden_dim2, hidden_dim3, latent_dim):
        super(Encoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_shape, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim3),
            nn.ReLU(),
            nn.Linear(hidden_dim3, latent_dim * 2)  # mean and variance
        )

    def forward(self, x):
        h = self.encoder(x)
        mu, log_var = h.chunk(2, dim=1)
        return mu, log_var


class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_shape):
        super(Decoder, self).__init__()

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim3),
            nn.ReLU(),
            nn.Linear(hidden_dim3, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, output_shape),
        )

    def forward(self, z):
        x_reconstructed = self.decoder(z)
        return x_reconstructed


class VAE(nn.Module):
    def __init__(self, input_shape, hidden_dim1, hidden_dim2, hidden_dim3, latent_dim):
        super(VAE, self).__init__()

        self.encoder = Encoder(input_shape, hidden_dim1, hidden_dim2, hidden_dim3, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim1, hidden_dim2, hidden_dim3, input_shape)

    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, log_var = self.encoder(x)
        z = self.reparameterize(mu, log_var)
        x_reconstructed = self.decoder(z)
        return x_reconstructed, mu, log_var

    
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split

import torch

combined_df = np.array(combined_arr)
combined_tensor = torch.from_numpy(combined_df)
combined_tensor = combined_tensor.permute(2, 0, 1).flatten(start_dim=1)

# Assuming combined_tensor is your data
# Convert the data to float32
dataset = TensorDataset(combined_tensor.float())

# Define the data loader
batch_size = 128  # adjust as necessary

# Split data into train, validation, and test
train_size = int(0.7 * len(dataset))  # 70% for training
valid_size = int(0.15 * len(dataset))  # 15% for validation
test_size = len(dataset) - train_size - valid_size  # 15% for testing

train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:

# Hyperparameters
# input_shape = combined_tensor.shape[0] * combined_tensor.shape[1]
input_shape = combined_tensor.shape[1]
hidden_dim1 = 1024  # modify as needed
hidden_dim2 = 128  # modify as needed
hidden_dim3 = 12  # modify as needed
latent_dim = 2  # modify as needed
lr = 5e-4  # learning rate
n_epochs = 300  # modify as needed
beta = 0.2


# Model, optimizer, and loss function
model = VAE(input_shape, hidden_dim1, hidden_dim2, hidden_dim3, latent_dim)

optimizer = optim.RAdam(model.parameters(), lr=lr)  # Make sure you're using the correct optimizer
loss_fn = nn.MSELoss()  # And the correct loss function


def train(epoch, model, optimizer, loss_fn, train_loader):
    model.train()
    train_loss = 0
    total_recon_loss = 0
    total_kl_div = 0
    for i, batch in enumerate(train_loader):  # using train_loader instead of dataloader
        batch_data = batch[0]  # get the data from the batch
        
        optimizer.zero_grad()

        # Flatten the data
        batch_data = batch_data.view(-1, input_shape)

        reconstructed_batch, mu, log_var = model(batch_data)

        # Loss: reconstruction loss + KL divergence
        recon_loss = loss_fn(reconstructed_batch, batch_data)
        kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        loss = recon_loss + beta*kl_divergence

        loss.backward()
        train_loss += loss.item()
        total_recon_loss += recon_loss.item()
        total_kl_div += kl_divergence.item()
        optimizer.step()

    avg_loss = train_loss / len(train_loader.dataset)
    avg_recon_loss = total_recon_loss / len(train_loader.dataset)
    avg_kl_div = total_kl_div / len(train_loader.dataset)
    print(f'====> Epoch: {epoch} Average loss: {avg_loss}, Recon Loss: {avg_recon_loss}, KL Div: {avg_kl_div}')

    return avg_loss, avg_recon_loss, avg_kl_div

    
import matplotlib.pyplot as plt

# Lists to store losses for each epoch
avg_losses = []
avg_recon_losses = []
avg_kl_divs = []

# Training
for epoch in range(1, n_epochs + 1):
    avg_loss, avg_recon_loss, avg_kl_div = train(epoch, model, optimizer, loss_fn, train_loader)
    avg_losses.append(avg_loss)
    avg_recon_losses.append(avg_recon_loss)
    avg_kl_divs.append(avg_kl_div)

# Plot the losses
plt.figure(figsize=(7,5))
plt.plot(avg_losses, label='Average Loss')
plt.plot(avg_recon_losses, label='Reconstruction Loss')
plt.plot(avg_kl_divs, label='KL Divergence')
plt.title('Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


# latent space plot

In [ ]:
# right

import numpy as np

# Get latent space representations for all data points
latent_vectors = []
model.eval()
with torch.no_grad():
    for batch in test_loader:
        batch_data = batch[0]
        batch_data = batch_data.view(batch_data.size(0), -1)
        mu, log_var = model.encoder(batch_data)
        z = model.reparameterize(mu, log_var)
        latent_vectors.append(z)

dance_latent_vectors = torch.cat(latent_vectors, dim=0).numpy()

# Plot the data in 2D latent space
plt.figure(figsize=(8, 6))
plt.scatter(dance_latent_vectors[:, 0], dance_latent_vectors[:, 1], alpha=0.5)
plt.xlabel('Latent Dimension 1')
plt.ylabel('Latent Dimension 2')
plt.title('2D Latent Space Visualization')
plt.show()


In [ ]:
# Assuming you have right_latent_vectors and front_latent_vectors numpy arrays

# Plot the data in 2D latent space with different colors
plt.figure(figsize=(8, 6))
plt.scatter(right_latent_vectors[:, 0], right_latent_vectors[:, 1], alpha=0.5, c='blue', label='Right Data')
plt.scatter(front_latent_vectors[:, 0], front_latent_vectors[:, 1], alpha=0.5, c='red', label='Front Data')
plt.scatter(bw_latent_vectors[:, 0], bw_latent_vectors[:, 1], alpha=0.5, c='black', label='Front Data')
plt.scatter(dance_latent_vectors[:, 0], dance_latent_vectors[:, 1], alpha=0.5, c='green', label='Dance Data')
plt.xlabel('Latent Dimension 1')
plt.ylabel('Latent Dimension 2')
plt.title('2D Latent Space Visualization')
plt.legend()  # Show the legend with label information
plt.show()


# plots  - front

In [ ]:
# UMAP

import umap

def visualize_latent_space_umap(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    reducer = umap.UMAP()
    embedding = reducer.fit_transform(latents)
    plt.figure(figsize=(7, 5))
    plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, s=3)
    plt.title("UMAP")
    plt.show()

visualize_latent_space_umap(model, train_loader)


In [ ]:
# pca

from sklearn.decomposition import PCA

def visualize_latent_space_pca(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(latents)
    plt.figure(figsize=(7, 5))
    plt.scatter(principalComponents[:, 0], principalComponents[:, 1], alpha=0.5, s=3)
    plt.title("PCA")
    plt.show()

visualize_latent_space_pca(model, train_loader)


In [ ]:
#t-sne

from sklearn.manifold import TSNE

def visualize_latent_space_tsne(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    tsne = TSNE(n_components=2)
    tsne_results = tsne.fit_transform(latents)
    plt.figure(figsize=(7, 5))
    plt.scatter(tsne_results[:, 0], tsne_results[:, 1], alpha=0.5, s=3)
    plt.title("t-SNE")
    plt.show()

visualize_latent_space_tsne(model, train_loader)


In [ ]:
# heatmap

import seaborn as sns

def visualize_latent_space_heatmap(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    plt.figure(figsize=(7, 5))
    sns.heatmap(latents)
    plt.title("Heatmap")
    plt.show()

visualize_latent_space_heatmap(model, train_loader)


# plots  - bw

In [ ]:
# UMAP

import umap

def visualize_latent_space_umap(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    reducer = umap.UMAP()
    embedding = reducer.fit_transform(latents)
    plt.figure(figsize=(7, 5))
    plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, s=3)
    plt.title("UMAP")
    plt.show()

visualize_latent_space_umap(model, train_loader)


In [ ]:
# pca

from sklearn.decomposition import PCA

def visualize_latent_space_pca(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(latents)
    plt.figure(figsize=(7, 5))
    plt.scatter(principalComponents[:, 0], principalComponents[:, 1], alpha=0.5, s=3)
    plt.title("PCA")
    plt.show()

visualize_latent_space_pca(model, train_loader)


In [ ]:
#t-sne

from sklearn.manifold import TSNE

def visualize_latent_space_tsne(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    tsne = TSNE(n_components=2)
    tsne_results = tsne.fit_transform(latents)
    plt.figure(figsize=(7, 5))
    plt.scatter(tsne_results[:, 0], tsne_results[:, 1], alpha=0.5, s=3)
    plt.title("t-SNE")
    plt.show()

visualize_latent_space_tsne(model, train_loader)


In [ ]:
# heatmap

import seaborn as sns

def visualize_latent_space_heatmap(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    plt.figure(figsize=(7, 5))
    sns.heatmap(latents)
    plt.title("Heatmap")
    plt.show()

visualize_latent_space_heatmap(model, train_loader)


# plots  - dance

In [ ]:
# UMAP

import umap

def visualize_latent_space_umap(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    reducer = umap.UMAP()
    embedding = reducer.fit_transform(latents)
    plt.figure(figsize=(7, 5))
    plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, s=3)
    plt.title("UMAP")
    plt.show()

visualize_latent_space_umap(model, train_loader)


In [ ]:
# pca

from sklearn.decomposition import PCA

def visualize_latent_space_pca(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(latents)
    plt.figure(figsize=(7, 5))
    plt.scatter(principalComponents[:, 0], principalComponents[:, 1], alpha=0.5, s=3)
    plt.title("PCA")
    plt.show()

visualize_latent_space_pca(model, train_loader)


In [ ]:
#t-sne

from sklearn.manifold import TSNE

def visualize_latent_space_tsne(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    tsne = TSNE(n_components=2)
    tsne_results = tsne.fit_transform(latents)
    plt.figure(figsize=(7, 5))
    plt.scatter(tsne_results[:, 0], tsne_results[:, 1], alpha=0.5, s=3)
    plt.title("t-SNE")
    plt.show()

visualize_latent_space_tsne(model, train_loader)


In [ ]:
# heatmap

import seaborn as sns

def visualize_latent_space_heatmap(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    plt.figure(figsize=(7, 5))
    sns.heatmap(latents)
    plt.title("Heatmap")
    plt.show()

visualize_latent_space_heatmap(model, train_loader)


# plots  - right

In [ ]:
# UMAP

import umap

def visualize_latent_space_umap(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    reducer = umap.UMAP()
    embedding = reducer.fit_transform(latents)
    plt.figure(figsize=(7, 5))
    plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, s=3)
    plt.title("UMAP")
    plt.show()

visualize_latent_space_umap(model, train_loader)


In [ ]:
# pca

from sklearn.decomposition import PCA

def visualize_latent_space_pca(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(latents)
    plt.figure(figsize=(7, 5))
    plt.scatter(principalComponents[:, 0], principalComponents[:, 1], alpha=0.5, s=3)
    plt.title("PCA")
    plt.show()

visualize_latent_space_pca(model, train_loader)


In [ ]:
#t-sne

from sklearn.manifold import TSNE

def visualize_latent_space_tsne(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    tsne = TSNE(n_components=2)
    tsne_results = tsne.fit_transform(latents)
    plt.figure(figsize=(7, 5))
    plt.scatter(tsne_results[:, 0], tsne_results[:, 1], alpha=0.5, s=3)
    plt.title("t-SNE")
    plt.show()

visualize_latent_space_tsne(model, train_loader)


In [ ]:
# heatmap

import seaborn as sns

def visualize_latent_space_heatmap(model, data_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            batch_data = batch[0]
            mu, _ = model.encoder(batch_data.view(batch_data.size(0), -1))
            latents.append(mu)
    latents = torch.cat(latents).numpy()
    plt.figure(figsize=(7, 5))
    sns.heatmap(latents)
    plt.title("Heatmap")
    plt.show()

visualize_latent_space_heatmap(model, train_loader)


# see the output

In [ ]:
def test_model(model, test_loader):
    model.eval()
    with torch.no_grad():
        for i, batch in enumerate(test_loader):
            batch_data = batch[0]  # get the data from the batch
            batch_data = batch_data.view(batch_data.size(0), -1)  # flatten the data
            reconstructed_batch, _, _ = model(batch_data)  # get reconstructed data from the model

            # Convert tensors to numpy arrays for use in pandas
            original_data = batch_data.detach().cpu().numpy()
            reconstructed_data = reconstructed_batch.detach().cpu().numpy()

            # Convert to dataframes
            original_df = pd.DataFrame(original_data)
            reconstructed_df = pd.DataFrame(reconstructed_data)

            if i == 0:  # for the first iteration, create the dataframes
                all_original_df = original_df
                all_reconstructed_df = reconstructed_df
            else:  # for subsequent iterations, append to the existing dataframes
                all_original_df = pd.concat([all_original_df, original_df])
                all_reconstructed_df = pd.concat([all_reconstructed_df, reconstructed_df])
    
    return all_original_df, all_reconstructed_df

# Call the function after training
original_df, reconstructed_df = test_model(model, test_loader)


In [ ]:
original_df, reconstructed_df = test_model(model, test_loader)

In [ ]:
original_df

In [ ]:
reconstructed_df

In [ ]:
# Get the first row from the dataframe
first_row = reconstructed_df.iloc[0]

# Reshape it to (1000, 10)
reshaped_array = np.reshape(first_row.values, (1000, 10))

# Convert it back to a dataframe
reshaped_df = pd.DataFrame(reshaped_array)
recon_combined_tensor = torch.tensor(reshaped_df.values)


In [ ]:
replay(reshaped_df)

In [ ]:
# Get the first row from the dataframe
first_row = original_df.iloc[0]

# Reshape it to (1000, 10)
reshaped_array = np.reshape(first_row.values, (1000, 10))

# Convert it back to a dataframe
reshaped_df = pd.DataFrame(reshaped_array)
ori_combined_tensor = torch.tensor(reshaped_df.values)


In [ ]:
replay(reshaped_df)

In [ ]:
recon_combined_tensor-ori_combined_tensor

In [ ]:
# Flatten the tensors
flattened_ori = ori_combined_tensor.flatten()
flattened_recon = recon_combined_tensor.flatten()

# Calculate the Euclidean distance
euclidean_distance = torch.norm(flattened_ori - flattened_recon)


print(euclidean_distance)


# VRNN

In [ ]:
import torch
from torch import nn

class VRNN(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim, n_layers, dropout=0.):
        super(VRNN, self).__init__()

        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim
        self.n_layers = n_layers

        self.encoder = nn.LSTM(x_dim + h_dim, z_dim, n_layers, dropout=dropout)
        self.decoder = nn.LSTM(z_dim + h_dim, x_dim, n_layers, dropout=dropout)
        self.hidden_to_hidden = nn.Linear(z_dim, h_dim)

    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = torch.zeros(self.n_layers, x.size(1), self.h_dim).to(x.device)

        for t in range(x.size(0)):
            _, h = self.encoder(torch.cat([x[t], h[-1]], -1).unsqueeze(0), h)
            mu, log_var = h.chunk(2, -1)
            z = self.reparameterize(mu, log_var)
            _, h = self.decoder(torch.cat([z, h[-1]], -1).unsqueeze(0), h)
            h = self.hidden_to_hidden(h)

        return h


In [ ]:
# Define the data loader
batch_size = 512  # adjust as necessary
seq_length = 1000  # sequence length

# Split data into train, validation, and test
train_size = int(0.7 * len(dataset))  # 70% for training
valid_size = int(0.15 * len(dataset))  # 15% for validation
test_size = len(dataset) - train_size - valid_size  # 15% for testing

train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Model, optimizer, and loss function
model = VRNN(x_dim=10, h_dim=128, z_dim=20, n_layers=2, dropout=0.2)

optimizer = optim.Adam(model.parameters(), lr=lr)  # Make sure you're using the correct optimizer
loss_fn = nn.MSELoss()  # And the correct loss function

# Training
for epoch in range(1, n_epochs + 1):
    for i, batch in enumerate(train_loader):  # using train_loader instead of dataloader
        batch_data = batch  # get the data from the batch
        
        optimizer.zero_grad()

        h = model(batch_data)

        # Loss: reconstruction loss + KL divergence
        recon_loss = loss_fn(h, batch_data)
        loss = recon_loss  # modify this line to include KL divergence

        loss.backward()
        optimizer.step()

    print(f'====> Epoch: {epoch} Loss: {loss.item()}')


In [ ]:
for i, batch in enumerate(train_loader):
    batch_data = batch
    print(batch_data.shape)
    break


# render with action

In [ ]:

import torch

# Convert combined_arr to PyTorch Tensor
# combined_tensor = torch.from_numpy(combined_arr)

# Print the shape of combined_tensor
print(combined_tensor.shape)

import matplotlib.pyplot as plt
from IPython import display
import gym


def replay(combined_data):

    action_sp = combined_data.iloc[:, :2]
    obs_sp = combined_data.iloc[:, 2:]

    env = gym.make('Swimmer-v3', render_mode = 'human')

    # Iterate through the rows
    for i in range(len(action_sp)):
        # Get the i-th row
        action = action_sp.iloc[i]
        observation = obs_sp.iloc[i]
        print(action)

        # If this is the first iteration, set the environment state to the given observation
        # Note: This assumes that the observation you've stored is the entire state that can be set with `env.reset()`
        # If this is not the case, you cannot simply set the environment state to the observation
        if i == 0:
            env.reset()  # We ignore the initial observation returned by `reset`

        # Apply the action
        next_observation, reward, done, trunc, info = env.step(action)
        # Render the environment
        env.render()
        # If you want to slow down each step for viewing, you can use time.sleep
        # time.sleep(0.01)

    # Close the environment
    env.close()


In [ ]:
replay()